In [27]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### 도구(tool)를 정의

In [28]:
from langchain.tools import tool

@tool
def add_function(a: float, b: float) -> float:
    """Adds two numbers together"""

    return a + b

`add_function` 을 사용하여 덧셈 계산을 수행하는 agent를 정의

In [29]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

In [30]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

In [31]:
tools = [add_function]                     # 도구 정의

In [32]:
llm = ChatOpenAI(                   
    api_key=key, 
    model='gpt-4o-mini', 
    temperature=0
)

In [33]:
prompt = ChatPromptTemplate.from_messages(  # 프롬프트
    [
        (
            'system',
            'You are a helpful assistant.',
        ),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [34]:
agent = create_tool_calling_agent(llm, tools, prompt)       # 에이전트

In [35]:
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True
)

### **AgentExecutor의 iter()**

AgentExecutor의 실행 과정을 단계별로 반복할 수 있게 해주는 반복자(iterator)를 생성합니다. 
<br><br>

에이전트가 최종 출력에 도달하기까지 거치는 단계들을 순차적으로 접근할 수 있는 AgentExecutorIterator 객체를 반환합니다.
<br><br>

`iter()` 에 질문을 넣게 되면 중간 단계라는 것을 받아올 수 있습니다. <br>
`setp.get()` 으로 결과를 action, value로 구분해서 받을 수 있습니다. <br>
action에서 `action.tool`로 도구(tool)이름을 비교해서 도구를 호출 했었으면 시행결과가 value에 저장됩니다.<br> 
그런다음 사용자가 `input()`를 통해서 계속 진행할 것인지 물어보게 되고 답변에 따라서 계속 진행 또는 멈추게 됩니다.<br>
<br>

`'114.5 + 121.2 + 34.2 + 110.1'` 덧셈 계산을 수행하기 위해서는 단계별로 계산이 수행되게 됩니다. <br>

1. 114.5 + 121.2 = 235.7
2. 235.7 + 34.2 = 270.9
3. 270.9 + 110.1 = 381.0
<br>

이러한 계산 과정을 단계별로 살펴볼 수 있습니다.
<br>

단계별로 계산 결과를 사용자에게 보여주고, 사용자가 계속 진행할지 묻습니다. (Human-in-the-loop) <br>

사용자가 'y'가 아닌 다른 입력을 하면 반복 중단됩니다.

In [36]:
question = "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"

for step in agent_executor.iter({'input': question}):
    print(step)



> Entering new None chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3{'intermediate_step': [(ToolAgentAction(tool='add_function', tool_input={'a': 114.5, 'b': 121.2}, log="\nInvoking: `add_function` with `{'a': 114.5, 'b': 121.2}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Bc0KLP3coJIGJLyzEJV1kbhR', 'function': {'arguments': '{"a": 114.5, "b": 121.2}', 'name': 'add_function'}, 'type': 'function'}, {'index': 1, 'id': 'call_4lXot0AoSy0QErnOD4uPqC76', 'function': {'arguments': '{"a": 34.2, "b": 110.1}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_00428b782a'}, id='run-29f24ba2-932c-447a-bfe5-64dbd85a804f', tool_calls=[{'name': 'add_function', 'args': {'a': 114.5, 'b': 121.2}, 'id': 'call_Bc0KLP3coJIGJLyzEJV1kbhR', 'typ

In [37]:
question = "114.5 + 121.2 + 34.2 + 110.1 의 계산 결과는?"

for step in agent_executor.iter({'input': question}):       # input 에 질문을 넣고 실행
    if output := step.get('intermediate_step'):             # intermediate_step 중간 단계를 얻는다
        action, value = output[0]                           # 결과를 action, value로 구분해서 넣는다
        
        print()
        print('=' * 50)
        print(f'\n[output]: {output}')
        print(f'action : {action}')
        print(f'value  : {value}')
        print('=' * 50)



> Entering new None chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3

[output]: [(ToolAgentAction(tool='add_function', tool_input={'a': 114.5, 'b': 121.2}, log="\nInvoking: `add_function` with `{'a': 114.5, 'b': 121.2}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_tKvu88xBfDQtYiaUT0bDx5bd', 'function': {'arguments': '{"a": 114.5, "b": 121.2}', 'name': 'add_function'}, 'type': 'function'}, {'index': 1, 'id': 'call_xn33agUmvCcRvvsKTPA1x1Cg', 'function': {'arguments': '{"a": 34.2, "b": 110.1}', 'name': 'add_function'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_13eed4fce1'}, id='run-19cfea86-91a6-4a52-b125-2fb87a4ac07c', tool_calls=[{'name': 'add_function', 'args': {'a': 114.5, 'b': 121.2}, 'id': 'call_tKvu88xBfDQtYiaUT0bDx5bd', 'type': 'tool_

In [38]:
for step in agent_executor.iter({'input': question}):       # input 에 질문을 넣고 실행
    if output := step.get('intermediate_step'):             # intermediate_step 중간 단계를 얻는다
        action, value = output[0]                           # 결과를 action, value로 구분해서 넣는다

        # print(f'action : {action}')
        # print(f'value  : {value}')
        
        if action.tool == 'add_function':                                   # action.tool 의 값이 add_function 이면 add_function 도구를 호출했다          
            print(f'Tool Name: {action.tool}, 실행 결과: {value}')          # Tool 실행 결과 출력
            
        _continue = input("계속 진행하시겠습니까? (y/n)?:\n") or "Y"        # 사용자에게 계속 진행할지 질문한다
        
        if _continue.lower() != "y":                                        # 사용자가 'y'가 아닌 다른 입력을 하면 반복 중단
            break



> Entering new None chain...

Invoking: `add_function` with `{'a': 114.5, 'b': 121.2}`


235.7
Invoking: `add_function` with `{'a': 34.2, 'b': 110.1}`


144.3Tool Name: add_function, 실행 결과: 235.7

Invoking: `add_function` with `{'a': 235.7, 'b': 144.3}`


380.0Tool Name: add_function, 실행 결과: 380.0
114.5 + 121.2 + 34.2 + 110.1의 계산 결과는 380.0입니다.

> Finished chain.
